In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
        self.len = len(self.file_names)
        if self.train:
            self.classes_mapping=self.get_classes()
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        file_name=self.file_names[index]
        image_data=self.pil_loader(self.file_path+"/"+file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            file_name_splitted=file_name.split("_")
            Y1 = self.classes_mapping[file_name_splitted[0]]
            Y2 = self.classes_mapping[file_name_splitted[1]]
            z1,z2=torch.zeros(10),torch.zeros(10)
            z1[Y1-10],z2[Y2]=1,1
            label=torch.stack([z1,z2])

            return image_data, label

        else:
            return image_data, file_name
          
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

      
    def get_classes(self):
        classes=[]
        for name in self.file_names:
            name_splitted=name.split("_")
            classes.extend([name_splitted[0],name_splitted[1]])
        classes=list(set(classes))
        classes_mapping={}
        for i,cl in enumerate(sorted(classes)):
            classes_mapping[cl]=i
        return classes_mapping
    

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()

In [4]:
import importlib
if importlib.util.find_spec('mlflow') is None:
  !pip install mlflow
import mlflow
import mlflow.pytorch

     |████████████████████████████████| 12.0MB 255kB/s 
     |████████████████████████████████| 286kB 35.6MB/s 
     |████████████████████████████████| 6.0MB 41.6MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 1.1MB 32.1MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 460kB 39.9MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 153kB 39.1MB/s 
     |████████████████████████████████| 122kB 42.6MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
     |████████████████████████████████| 2.7MB 38.4MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 204kB 46.2MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Created wheel for alembic: filename=alembic

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
!mkdir -p hin_classifier
!unzip -nq "/content/drive/My Drive/hin_classifier/train.zip" -d hin_classifier
!unzip -nq "/content/drive/My Drive/hin_classifier/test.zip" -d hin_classifier

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [8]:
transform = transforms.Compose([
    transforms.ToTensor()])

In [9]:
full_data = VowelConsonantDataset("hin_classifier/train",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=60, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=60, shuffle=True)

In [10]:
test_data = VowelConsonantDataset("hin_classifier/test",train=False,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=60,shuffle=False)

In [11]:
def label_tensor(actual_labels):
    return torch.LongTensor([torch.max(labels, dim = -1)[1].item() for labels in actual_labels])

In [13]:
class Params(object):
    def __init__(self, batch_size, epochs, seed, log_interval):
        self.batch_size = batch_size
        self.epochs = epochs
        self.seed = seed
        self.log_interval = log_interval

args = Params(256, 4, 0, 20)

In [12]:
class FeedForwardNetwork(nn.Module):
    def __init__(self): 
        super(FeedForwardNetwork, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(12288, 2),  # 64 x 64 x 3 = 12288
            nn.ReLU(),
            nn.Linear(2, 2),
            nn.ReLU(),
            nn.Linear(2, 10)
        )
             
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [20]:
def train(network, optimise, epochs, compute_loss):
    
  network.train()
  for epoch in range(epochs):   
    for batch_id, data in enumerate(train_loader):
      inputs, labels = data
              
      optimise.zero_grad()
      inputs = inputs.to(device)
      outputs = network(inputs)
                    
      labels_reshaped = labels.permute(1,0,2)
      vow_labels = label_tensor(labels_reshaped[0])
      con_labels = label_tensor(labels_reshaped[1])
      vow_labels = vow_labels.to(device)
      con_labels = con_labels.to(device)

      loss = compute_loss(outputs, vow_labels)
      loss += compute_loss(outputs, con_labels)


      loss.backward()
      optimise.step()
        
      if batch_id % args.log_interval == 0:
        pos = epoch * len(train_loader) + batch_id
        mlflow.log_metric('train_loss', loss.data.item()/len(inputs)*1000)
            
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
                epoch, batch_id * len(inputs), len(train_loader.dataset),
                100. * batch_id / len(train_loader), loss.data.item()))

In [16]:
max_epochs = 16 if torch.cuda.is_available() else 5

In [21]:
%%time
ffn = FeedForwardNetwork().to(device)
loss_epoch_arr = []
train(ffn, optim.Adam(ffn.parameters()), max_epochs,nn.CrossEntropyLoss())

Train Epoch: 0 [0/9000 (0%)]	Loss: 4.653
Train Epoch: 0 [1200/9000 (13%)]	Loss: 4.772
Train Epoch: 0 [2400/9000 (27%)]	Loss: 4.662
Train Epoch: 0 [3600/9000 (40%)]	Loss: 4.855
Train Epoch: 0 [4800/9000 (53%)]	Loss: 4.629
Train Epoch: 0 [6000/9000 (67%)]	Loss: 4.704
Train Epoch: 0 [7200/9000 (80%)]	Loss: 4.682
Train Epoch: 0 [8400/9000 (93%)]	Loss: 4.684
Train Epoch: 1 [0/9000 (0%)]	Loss: 4.694
Train Epoch: 1 [1200/9000 (13%)]	Loss: 4.706
Train Epoch: 1 [2400/9000 (27%)]	Loss: 4.684
Train Epoch: 1 [3600/9000 (40%)]	Loss: 4.609
Train Epoch: 1 [4800/9000 (53%)]	Loss: 4.709
Train Epoch: 1 [6000/9000 (67%)]	Loss: 4.637
Train Epoch: 1 [7200/9000 (80%)]	Loss: 4.676
Train Epoch: 1 [8400/9000 (93%)]	Loss: 4.644
Train Epoch: 2 [0/9000 (0%)]	Loss: 4.666
Train Epoch: 2 [1200/9000 (13%)]	Loss: 4.617
Train Epoch: 2 [2400/9000 (27%)]	Loss: 4.610
Train Epoch: 2 [3600/9000 (40%)]	Loss: 4.601
Train Epoch: 2 [4800/9000 (53%)]	Loss: 4.662
Train Epoch: 2 [6000/9000 (67%)]	Loss: 4.611
Train Epoch: 2 [7200/9